In [1]:
import os
from datetime import datetime
import shutil
import yaml
from tqdm import tqdm
import torch
import numpy as np
import sys
%matplotlib widget
import matplotlib as mpl
from matplotlib import pyplot as plt 


import scipy.io as sio
import logging
from transformers import GPT2Model, GPT2Tokenizer, GPT2Config
from collections import namedtuple, defaultdict
import transformers as tr
from transformers import pipeline
import pandas as pd 
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise 
from itertools import cycle, islice
import scipy as sc
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import bcolz
import pickle

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
save_path="/Users/eghbalhosseini/MyData/semantic_proj/"
analysis_path="/Users/eghbalhosseini/MyData/semantic_proj/analysis/"
analysis_type='compare_NN_empedding_with_glove'
model_type="gpt2-xl"
LAYER_COUNT = 48
FEATURE_COUNT = 1024
access_rights = 0o755
try:
    os.mkdir(save_path,access_rights)
except OSError:
    print('Creation of %s failed\n'%save_path)
model_path=os.path.join(save_path,model_type)
try:
    os.mkdir(model_path,access_rights)
except OSError:
    print('Creation of %s failed\n'%model_path)
    
try:
    os.mkdir(analysis_path,access_rights)
except OSError:
    print('Creation of %s failed\n'%analysis_path)
model_path=os.path.join(analysis_path,analysis_type)
try:
    os.mkdir(model_path,access_rights)
except OSError:
    print('Creation of %s failed\n'%model_path)

Creation of /Users/eghbalhosseini/MyData/semantic_proj/ failed

Creation of /Users/eghbalhosseini/MyData/semantic_proj/gpt2-xl failed

Creation of /Users/eghbalhosseini/MyData/semantic_proj/analysis/ failed

Creation of /Users/eghbalhosseini/MyData/semantic_proj/analysis/compare_NN_empedding_with_glove failed



In [3]:
def match_tokenized_to_untokenized_gpt2(tokenized_sent, untokenized_sent):
    mapping = defaultdict(list)
    untokenized_sent_index = 0
    tokenized_sent_index = 0
    while (untokenized_sent_index < len(untokenized_sent) and
        tokenized_sent_index < len(tokenized_sent)):
      while (tokenized_sent_index + 1 < len(tokenized_sent) and
          not(tokenized_sent[tokenized_sent_index + 1].startswith("Ġ"))):
        mapping[untokenized_sent_index].append(tokenized_sent_index)
        tokenized_sent_index += 1
      mapping[untokenized_sent_index].append(tokenized_sent_index)
      untokenized_sent_index += 1
      tokenized_sent_index += 1
    return mapping

In [4]:
# configure the gpt2 model 
config=GPT2Config.from_pretrained(model_type)
config.output_hidden_states=True
# make model from config
model=GPT2Model(config)
model.from_pretrained(model_type)
device = torch.device("cpu")
model.to(device)
# 
tokenizer = GPT2Tokenizer.from_pretrained(model_type)

In [5]:
SEMANTIC_DAT_DIR='data_from_semantic_paper'
SEMANTIC_DAT_FILE='GabeDimensions.xlsx'
Dimensions = pd.read_excel(os.path.join(save_path,SEMANTIC_DAT_DIR,SEMANTIC_DAT_FILE),sheet_name='Dimensions')
Domains = pd.read_excel(os.path.join(save_path,SEMANTIC_DAT_DIR,SEMANTIC_DAT_FILE),sheet_name='Domains top 25',skiprows=1)
DIMENSIONSxDOMAINS=pd.read_excel(os.path.join(save_path,SEMANTIC_DAT_DIR,SEMANTIC_DAT_FILE),sheet_name='DIMENSIONSxDOMAINS',skiprows=0)
# 
DIMENSIONSxDOMAINS=DIMENSIONSxDOMAINS.drop(DIMENSIONSxDOMAINS.index[18:],axis=0)
DIMENSIONSxDOMAINS=DIMENSIONSxDOMAINS.drop(['Unnamed: 21','Domain Usefulness','Number of Greens'],axis=1)
# 
glove_path='/Users/eghbalhosseini/MyData/semantic_proj/GLoVe/'
vectors = bcolz.open(f'{glove_path}/42B.300.dat')[:]
words = pickle.load(open(f'{glove_path}/42B.300_words.pkl', 'rb'))
word2idx = pickle.load(open(f'{glove_path}/42B.300_idx.pkl', 'rb'))
glove = {w: vectors[word2idx[w]] for w in words}


In [7]:
plt.ioff()

In [42]:
a.size

0

In [ ]:
for domain in tqdm(Domains.columns[:-1]):
    # 1st: get the layerwise representation for each layer of the network
    lineList=Domains[domain]
    corr_entry=[type(x)==str for x in lineList]
    lineList=lineList[corr_entry]
    representation_arr = np.zeros((len(lineList),2), dtype=np.object)
    for index, line in enumerate(lineList):
        untokenized_word = line.strip().split()
        tokenized_word = tokenizer.tokenize(line,add_prefix_space=True)
        untok_tok_mapping = match_tokenized_to_untokenized_gpt2(tokenized_word, untokenized_word)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_word)
        segment_ids = [1 for x in tokenized_word]
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segment_ids])
        with torch.no_grad():
            encoded_layers = model(tokens_tensor)
            last_hidden_state=encoded_layers[0]
            hidden_output=encoded_layers[2]
            layer_wise_feature=[torch.stack([torch.mean(F[0,untok_tok_mapping[i][0]:untok_tok_mapping[i][-1]+1,:], dim=0) for i in range(len(untokenized_word))], dim=0) for F in hidden_output]
            layer_wise_reperensetation= [R.view(1,*R.size()) for R in layer_wise_feature ]
            rep_arr=[np.squeeze(x.numpy()) for x in layer_wise_reperensetation]
            representation_arr[index,0]=' '.join(untokenized_word)
            representation_arr[index,1]=rep_arr
    # 2nd: get domains for which the dimension is reasonable
    
    a=np.argwhere(DIMENSIONSxDOMAINS['Unnamed: 0']==domain)
    if a.size>0:
        lineList=DIMENSIONSxDOMAINS.iloc[a.squeeze()][0:]
        # threshold
        lineList==3
        feature_domain_names=DIMENSIONSxDOMAINS.columns[np.argwhere(lineList==3)]
        # 3rd extract feature domains and project for both NN and glove
        for k,name in enumerate(feature_domain_names):
            feat=np.argwhere(Dimensions.Dimension==name)
            b=Dimensions.iloc[feat[0]].iloc[0][1:]
            lineList=b
            dimensions_arr = np.zeros((len(lineList),2), dtype=np.object)
            for index, line in enumerate(lineList):
                untokenized_word = line.strip().split()
                tokenized_word = tokenizer.tokenize(line,add_prefix_space=True)
                untok_tok_mapping = match_tokenized_to_untokenized_gpt2(tokenized_word, untokenized_word)
                indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_word)
                segment_ids = [1 for x in tokenized_word]
                tokens_tensor = torch.tensor([indexed_tokens])
                segments_tensors = torch.tensor([segment_ids])
                with torch.no_grad():
                    encoded_layers = model(tokens_tensor)
                    last_hidden_state=encoded_layers[0]
                    hidden_output=encoded_layers[2]
                    layer_wise_feature=[torch.stack([torch.mean(F[0,untok_tok_mapping[i][0]:untok_tok_mapping[i][-1]+1,:], dim=0) for i in range(len(untokenized_word))], dim=0) for F in hidden_output]
                    layer_wise_reperensetation= [R.view(1,*R.size()) for R in layer_wise_feature ]
                    rep_arr=[np.squeeze(x.numpy()) for x in layer_wise_reperensetation]
                    dimensions_arr[index,0]=' '.join(untokenized_word)
                    dimensions_arr[index,1]=rep_arr

        # 4th : project the data onto the representation axis
            dim_positive=dimensions_arr[0:3]
            dim_negative=dimensions_arr[3:]
            sem_vec=[[np.stack(y[1])-np.stack(x[1]) for y in dim_negative] for x in dim_positive]
            sem_vec_1=[np.stack(x) for x in sem_vec]
            sem_vec_2=np.concatenate(sem_vec_1,axis=0)
            sem_vec_ave=np.mean(sem_vec_2,axis=0)
        # 5th : get the word projection onto the dimension axis across layers
            NN_resp_projection=[]
            NN_resp_names=[]
            for i, x in enumerate(representation_arr):
                NN_response=np.stack(x[1]);
                a=np.dot(sem_vec_ave,NN_response.transpose())
                NN_resp_projection.append(np.diag(a))
                NN_resp_names.append(x[0])

        # 6th get glove embedding
        # 6.1 get dim vectors
            dimensions_arr=[glove[x] for x in lineList]
            dim_positive=dimensions_arr[0:3]
            dim_negative=dimensions_arr[3:]

            sem_vec=[[np.stack(y)-np.stack(x) for y in dim_negative] for x in dim_positive]
            sem_vec_1=[np.stack(x) for x in sem_vec]
            sem_vec_2=np.concatenate(sem_vec_1,axis=0)
            glove_sem_vec_ave=np.mean(sem_vec_2,axis=0)
        # 6.2 get domains
            lineList=Domains[domain]
            corr_entry=[type(x)==str for x in lineList]
            lineList=lineList[corr_entry]
            Glove_rep_arr = np.zeros((len(lineList),2), dtype=np.object)
            for index, line in enumerate(lineList):
                Glove_rep_arr[index,0]=line
                Glove_rep_arr[index,1]=glove[line]
        # 6.3 projection
            Glove_resp_projection=[]
            Glove_resp_names=[]
            for i, x in enumerate(Glove_rep_arr):
                Glove_response=x[1]
                a=np.dot(glove_sem_vec_ave,Glove_response.transpose())
                Glove_resp_projection.append(a)
                Glove_resp_names.append(x[0])
            NN_rep_proj_mat=np.stack(NN_resp_projection).transpose()
            plt.figure(figsize=[20,25])
            x=np.asarray(Glove_resp_projection).reshape(-1, 1)
            for i, y in enumerate(NN_rep_proj_mat):
                regr = linear_model.LinearRegression()
                y=y.reshape(-1, 1)
                regr.fit(x, y)
                y_pred = regr.predict(x)
                plt.subplot(10,5,i+1)
                plt.scatter(x,y,s=5,c=[0,0,0])
                plt.plot(x, y_pred, color='blue', linewidth=3)
                a=r2_score(y,y_pred)
                s=f"{a:.3f}"
                plt.title('R squared: '+s)
            plt.savefig(model_path+'/'+model_type+'_vs_GloVe_'+domain+'_'+name+'.pdf', format='pdf',bbox_inches='tight')
            plt.close()


0                may
1          christmas
2              march
3               june
4              april
5               july
6             august
7          september
8            october
9           november
10      thanksgiving
11          december
12         halloween
13           january
14            easter
15          february
16              lent
17          hanukkah
18            advent
19          passover
20           ramadan
21    valentines-day
Name: Calendar, dtype: object

,Unnamed: 0,Size,Weight,Temperature,Time,Speed,Loudness,Location,Danger,Wetness,...,Wealth,Gender,Taste,Intelligence,Emotion,Quality,Valence,Arousal,Political,Religiosity


In [ ]:
feature_domain_names=DIMENSIONSxDOMAINS.columns[np.argwhere(lineList==3)]